In [122]:
import json
import csv
import nltk

states = ["ak","al","ar","az","ca","co","ct","de","fl","ga","hi","ia","id","il",
          "in","ks","ky","la","ma","md","me","mi","mn","mo","ms","mt","nc","nd","ne","nh",
          "nj","nm","nv","ny","oh","ok","or","pa","ri","sc","sd","tn","tx","ut","va","vt",
          "wa","wi","wv","wy"]

states_names = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
'CT': 'Connecticut', 'DE': 'Delaware', 'DC': 'District of Columbia', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois',
'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC':'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island',
'SC': 'South Carolina', 'SD': 'South Dakota', 'TN':'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia',
'WA': 'Washington','WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming', 'PR': 'Puerto Rico'}

afinnfile = open("AFINN-111.txt")
afinnDict = {} # initialize an empty dictionary
#scores_tuples = [line.decode('utf-8').split('\t') for line in afinnline]
#scores = dict((a,int(b)) for a,b in scores_tuples)

for line in afinnfile:
    term, score  = line.split("\t")  # The file is tab-delimited. "\t" means "tab character"
    afinnDict[term] = int(score)  # Convert the score to an integer.

def getState(data):
    if data["place"] != None and data["place"]["country_code"] == "US":
        state = str(data["place"]["full_name"]).lower().split(", ")
        
        if len(state) > 1:
            return state[1]

def isState(state):
    if state in states:
        return True
    return False 

def det_sent(tweet): #helper method for determining net sentiment, will use the sentiment score to appply scores to unscored words.
    terms_count = 0
    score = 0

    for word in tweet:
        stripped_word = word.lower().strip().replace('.','').replace('?','').replace('!','')

        if stripped_word in afinnDict.keys():
            terms_count += 1
            score += afinnDict[stripped_word]
        
    return terms_count, score

def analyze():
    file = "inp.txt"
    scored_tweets = []
    tweetsByStates = {}
       
    with open(file, "r", encoding='utf-16') as ins:
        
        for line in ins:          
            line = line.strip("'<>() ").replace('\'', '\"')
            line = line.replace("b\"{", "{")
            line = line.replace("}\"", "}")
            line = line.replace("\\\\", "\\")
        
            if len(line) > 1: ## to avoid empty lines 
                tweet = json.loads(line, encoding="utf-16")
           
                if "created_at" in tweet:
                    state = getState(tweet)

                    if isState(state):
                        if "text" in tweet:                           
                            tweetText = tweet["text"]
                            tokenizedTweet = nltk.word_tokenize(tweetText)
                            
                            score = 0
                            numberTerms = 0
                            
                            numberTerms, score = det_sent(tokenizedTweet)
                            
                            if numberTerms > 0:
                                t = [tweetText, state, score, numberTerms]
                                scored_tweets.append(t)
                                
                                if state in tweetsByStates:
                                    tweetsByStates[state].append(score)
                                else:
                                    tweetsByStates[state] = [score]
    print(tweetsByStates)
    
    with open('out.csv', mode='w',  newline='') as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(["state", "id", "meanSent"])
        i = 0
        mean = None
        for state in states_names:
            i+=1
            lowerCaseState = state.lower()
            if lowerCaseState in tweetsByStates:
                mean = sum(tweetsByStates[lowerCaseState])/len(tweetsByStates[lowerCaseState])
            else: mean = 0
            
            writer.writerow([states_names[state], i, mean])

In [123]:
analyze()

{'ca': [2, 2, 3], 'tx': [6, 7, -6, -3, -1, 2], 'al': [1], 'oh': [-10, 2, 3, 1, 0, -3], 'id': [2], 'ga': [-5, -4, -4], 'or': [2], 'fl': [1, -3, 3], 'ny': [3, -2, 3, -4], 'mn': [1, -5, 1], 'pa': [3], 'nj': [2], 'mi': [-2], 'nd': [-3], 'ok': [4], 'md': [1]}


In [49]:
import sys, json
struct = {}
try:
    try: #try parsing to dict
        dataform = str(response_json).strip("'<>() ").replace('\'', '\"')
        struct = json.loads(dataform)
    except:
        print(repr(resonse_json))
        print(sys.exc_info())


SyntaxError: unexpected EOF while parsing (<ipython-input-49-469586974c90>, line 9)